## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.3190,79.12,88,90,14.97,light rain
1,1,Busselton,AU,-33.6500,115.3333,47.08,86,0,8.08,clear sky
2,2,Qaanaaq,GL,77.4840,-69.3632,40.78,99,100,6.40,light rain
3,3,Punta Arenas,CL,-53.1500,-70.9167,35.62,96,100,1.25,overcast clouds
4,4,Provideniya,RU,64.3833,-173.3000,47.68,98,100,5.66,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 88


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kapaa,US,22.0752,-159.3190,79.12,88,90,14.97,light rain
6,6,Kavieng,PG,-2.5744,150.7967,83.23,75,48,16.51,scattered clouds
7,7,Olinda,BR,-8.0089,-34.8553,78.53,65,40,17.27,scattered clouds
15,15,Atuona,PF,-9.8000,-139.0333,77.70,79,31,16.28,scattered clouds
16,16,Kalemie,CD,-5.9475,29.1947,78.91,47,11,4.88,few clouds
17,17,Coahuayana,MX,18.7333,-103.6833,85.48,67,52,2.30,broken clouds
22,22,Turan,VN,16.0678,108.2208,87.85,65,20,3.44,few clouds
25,25,Mandal,IN,25.4500,74.5667,80.55,66,70,12.26,broken clouds
26,26,Russkaya Polyana,UA,49.4170,31.9201,77.34,43,0,6.08,clear sky
27,27,Korla,CN,41.7597,86.1469,83.97,25,98,7.78,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                243
City                   243
Country                243
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [7]:
clean_preferred_cities_df.count()

City_ID                243
City                   243
Country                243
Lat                    243
Lng                    243
Max Temp               243
Humidity               243
Cloudiness             243
Wind Speed             243
Current Description    243
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,79.12,light rain,22.0752,-159.3190,
6,Kavieng,PG,83.23,scattered clouds,-2.5744,150.7967,
7,Olinda,BR,78.53,scattered clouds,-8.0089,-34.8553,
15,Atuona,PF,77.70,scattered clouds,-9.8000,-139.0333,
16,Kalemie,CD,78.91,few clouds,-5.9475,29.1947,
17,Coahuayana,MX,85.48,broken clouds,18.7333,-103.6833,
22,Turan,VN,87.85,few clouds,16.0678,108.2208,
25,Mandal,IN,80.55,broken clouds,25.4500,74.5667,
26,Russkaya Polyana,UA,77.34,clear sky,49.4170,31.9201,
27,Korla,CN,83.97,overcast clouds,41.7597,86.1469,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   243
Country                243
Max Temp               243
Current Description    243
Lat                    243
Lng                    243
Hotel Name             243
dtype: int64

In [11]:
hotel_df = hotel_df.replace(r'^\s*$', np.NaN, regex=True)

In [12]:
# No empty rows for Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [13]:
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kapaa,US,79.12,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Kavieng,PG,83.23,scattered clouds,-2.5744,150.7967,Nusa Island Retreat
7,Olinda,BR,78.53,scattered clouds,-8.0089,-34.8553,Hotel Albergue de Olinda
15,Atuona,PF,77.70,scattered clouds,-9.8000,-139.0333,Villa Enata
16,Kalemie,CD,78.91,few clouds,-5.9475,29.1947,Mahobilali


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))